In [82]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
from scipy.stats.mstats import winsorize
from statsmodels.stats.sandwich_covariance import cov_hac

In [83]:
# =============================================================================
#   1. Functions
# =============================================================================

# Data cleaning: handling months with insufficient data
def handle_outlier_months(daily_rtn_df):
    for stock in daily_rtn_df.columns:
        monthly_count = daily_rtn_df[stock].resample('ME').count()
        months_to_nan = monthly_count[monthly_count < 7].dropna().index
        daily_rtn_df.loc[daily_rtn_df.index.to_period('M').isin(months_to_nan.to_period('M')), stock] = np.nan
    return daily_rtn_df

# Semibeta calculation
def semibeta(r_signed, m_signed, m_total):
    num_df = r_signed * m_signed.values
    denom_df = m_total ** 2
    sum_num_df = num_df.resample('ME').sum(min_count=1)
    sum_denom_df = denom_df.resample('ME').sum(min_count=1)
    semibeta_df = sum_num_df / sum_denom_df.values
    return semibeta_df

def semibeta_mix(r_signed, m_signed, m_total):
    num_df = r_signed * m_signed.values
    denom_df = m_total ** 2
    sum_num_df = num_df.resample('ME').sum(min_count=1)
    sum_denom_df = denom_df.resample('ME').sum(min_count=1)
    semibeta_df = -sum_num_df / sum_denom_df.values  # sign change
    return semibeta_df

# Function to winsorize a single column
def winsorize_col(series, lower_percentile, upper_percentile):
    return pd.Series(winsorize(series, limits=(lower_percentile, upper_percentile)), index=series.index)

# Semibeta summary statistics
def semibeta_sumstat(semibeta_df):
    cross_mean = semibeta_df.mean(axis=1)
    cross_std = semibeta_df.std(axis=1, ddof=1)
    cross_median = semibeta_df.median(axis=1)
    
    ts_mean = cross_mean.mean()
    ts_std = cross_std.mean()
    ts_median = cross_median.mean()
    
    return ts_mean, ts_std, ts_median

# Time-series mean of cross-sectional correlation
def average_cross_sec_corr(matrix1, matrix2):
    correlations = []
    for i in range(len(matrix1)):
        corr = matrix1.iloc[i].corr(matrix2.iloc[i], 'pearson')
        correlations.append(corr)
    return np.mean(correlations)

def newey_west_se(lambdas, nlags):
    results = sm.OLS(lambdas, np.ones(len(lambdas))).fit()
    nw_cov = cov_hac(results, nlags=nlags)
    nw_se = np.sqrt(np.diag(nw_cov))
    return nw_se

def newey_west_tstat(lambdas, mean_lambdas, nlags):
    nw_se = newey_west_se(lambdas, nlags)
    nw_tstat = mean_lambdas / nw_se
    p_values = 2 * (1 - norm.cdf(np.abs(nw_tstat)))
    return nw_tstat, p_values

In [84]:
# =============================================================================
#   X. Data loading
# =============================================================================

file_path = './data/kor/final/'
r = pd.read_csv(f'{file_path}KOSPI_r_daily_811_covid.csv')
# r_monthly = pd.read_csv(f'{file_path}KOSPI_r_monthly_811_paper.xlsx')
m = pd.read_csv(f'{file_path}KOSPI_m_daily_covid.csv')

In [85]:
# =============================================================================
#   X. Data cleaning
# =============================================================================

# Data cleaning (r)
col_name_r = r.iloc[7].tolist()
r = r.drop(r.index[0:13])
r.columns = col_name_r
r['Symbol'] = pd.to_datetime(r['Symbol'])
r.set_index('Symbol', inplace=True)
r.index.name = None
r = r.astype('float64')
r = r / 100

# # Data cleaning (r_monthly)
# col_name_r_monthly = r_monthly.iloc[7].dropna().tolist()
# r_monthly = r_monthly.drop(r_monthly.index[0:13])
# r_monthly.columns = col_name_r_monthly
# r_monthly['Symbol'] = pd.to_datetime(r_monthly['Symbol'])
# r_monthly.set_index('Symbol', inplace=True)
# r_monthly.index.name = None
# r_monthly = r_monthly.astype('float64')
# r_monthly = r_monthly / 100

# Data cleaning (m)
col_name_m = m.iloc[7].dropna().tolist()
m = m.drop(m.index[0:13])
m = m.iloc[:, :-1]
m.columns = col_name_m
m['Symbol'] = pd.to_datetime(m['Symbol'])
m.set_index('Symbol', inplace=True)
m.index.name = None
m = m.astype('float64')
m = m / 100

# Extract positive and negative returns
r_pos = r.mask(r < 0, 0)
r_neg = r.mask(r > 0, 0)
m_pos = m.mask(m < 0, 0)
m_neg = m.mask(m > 0, 0)

# Handle outlier months with less than 10 data points
r = handle_outlier_months(r)
m = handle_outlier_months(m)
r_pos = handle_outlier_months(r_pos)
r_neg = handle_outlier_months(r_neg)
m_pos = handle_outlier_months(m_pos)
m_neg = handle_outlier_months(m_neg)


/var/folders/bp/kjbh4_8964599ypdwfdp5b640000gn/T/ipykernel_11528/2048044915.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  r['Symbol'] = pd.to_datetime(r['Symbol'])
/var/folders/bp/kjbh4_8964599ypdwfdp5b640000gn/T/ipykernel_11528/2048044915.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  m['Symbol'] = pd.to_datetime(m['Symbol'])


In [86]:
# =============================================================================
#   2. Semibeta calculation
# =============================================================================

# Calculate monthly semibetas
beta_CAPM = semibeta(r, m, m)
beta_N = semibeta(r_neg, m_neg, m)
beta_P = semibeta(r_pos, m_pos, m)
beta_M_pos = semibeta_mix(r_neg, m_pos, m)
beta_M_neg = semibeta_mix(r_pos, m_neg, m)

# Winsorize semibetas at 1% and 99% levels
lower_percentile = 0.01
upper_percentile = 0.01
beta_CAPM = beta_CAPM.apply(winsorize_col, lower_percentile=lower_percentile, upper_percentile=upper_percentile)
beta_N = beta_N.apply(winsorize_col, lower_percentile=lower_percentile, upper_percentile=upper_percentile)
beta_P = beta_P.apply(winsorize_col, lower_percentile=lower_percentile, upper_percentile=upper_percentile)
beta_M_pos = beta_M_pos.apply(winsorize_col, lower_percentile=lower_percentile, upper_percentile=upper_percentile)
beta_M_neg = beta_M_neg.apply(winsorize_col, lower_percentile=lower_percentile, upper_percentile=upper_percentile)

# Semibeta summary statistics
sumstat_index = ['Mean', 'Std', 'Med']
sumstat_col = ['B', 'B_N', 'B_P', 'B_M+', 'B_M-']
sumstat = pd.DataFrame(index=sumstat_index, columns=sumstat_col)
sumstat['B'] = semibeta_sumstat(beta_CAPM)
sumstat['B_N'] = semibeta_sumstat(beta_N)
sumstat['B_P'] = semibeta_sumstat(beta_P)
sumstat['B_M+'] = semibeta_sumstat(beta_M_pos)
sumstat['B_M-'] = semibeta_sumstat(beta_M_neg)

# Correalation between semibetas (simple element-wise)
betas = [beta_CAPM, beta_N, beta_P, beta_M_pos, beta_M_neg]
corr_matrix_label = ['B', 'B_N', 'B_P', 'B_M+', 'B_M-']
corr_matrix = pd.DataFrame(index=corr_matrix_label, columns=corr_matrix_label)

for i in range(len(betas)):
    for j in range(i, len(betas)):
        if i == j:
            corr_matrix.iloc[i, j] = 1.0
        else:
            flat1 = betas[i].values.flatten()
            flat2 = betas[j].values.flatten()
            combined_df = pd.DataFrame({'flat1': flat1, 'flat2': flat2})
            clean_df = combined_df.dropna()
            corr = np.corrcoef(clean_df['flat1'], clean_df['flat2'])[0, 1]
            # corr_matrix.iloc[i, j] = corr
            corr_matrix.iloc[j, i] = corr
            
# Output summary stats
print(sumstat)
print()
print(corr_matrix)
print()
print()

             B       B_N       B_P      B_M+      B_M-
Mean  0.967195  0.686311  0.538842  0.102022  0.155937
Std   0.642442  0.327676  0.350387  0.145789  0.273188
Med   0.964514  0.670893  0.478587  0.058249  0.075422

             B       B_N       B_P      B_M+ B_M-
B          1.0       NaN       NaN       NaN  NaN
B_N   0.627099       1.0       NaN       NaN  NaN
B_P   0.565486  0.033365       1.0       NaN  NaN
B_M+ -0.290446 -0.065968   0.24383       1.0  NaN
B_M- -0.439389  0.032286  0.090742  0.380278  1.0




In [87]:
# =============================================================================
#   X. Comparison with reference paper
# =============================================================================

# Reference paper sumstat
ref_index = ['Mean', 'Stdv', 'Med']
ref_data = {
    'B': [0.71, 0.71, 0.67],
    'B_N': [0.54, 0.41, 0.45],
    'B_P': [0.58, 0.46, 0.47],
    'B_M+': [0.21, 0.26, 0.13],
    'B_M-': [0.20, 0.27, 0.10]
}
ref_df = pd.DataFrame(index=ref_index, data=ref_data)

# Reference paper correlation matrix
ref_index2 = ['B', 'B_N', 'B_P', 'B_M+', 'B_M-']
ref_data2 = {
    'B': [1.00, 0.61, 0.58, -0.34, -0.34],
    'B_N': [np.nan, 1.00, 0.14, 0.04, 0.10],
    'B_P': [np.nan, np.nan, 1.00, 0.21, 0.19],
    'B_M+': [np.nan, np.nan, np.nan, 1.00, 0.38],
    'B_M-': [np.nan, np.nan, np.nan, np.nan, 1.00]
}
ref_matrix = pd.DataFrame(index=ref_index2, data=ref_data2)

print('Sig diff between ref results')
stat_diff = ref_df - sumstat
stat_diff = stat_diff.where(stat_diff.abs() > 0.1, np.nan)
print(stat_diff)

print()
corr_diff = ref_matrix - corr_matrix
corr_diff = corr_diff.where(corr_diff.abs() > 0.05, np.nan)
print(corr_diff)

Sig diff between ref results
             B       B_N  B_P      B_M+  B_M-
Mean -0.257195 -0.146311  NaN  0.107978   NaN
Med  -0.294514 -0.220893  NaN       NaN   NaN
Std        NaN       NaN  NaN       NaN   NaN
Stdv       NaN       NaN  NaN       NaN   NaN

             B       B_N       B_P B_M+ B_M-
B          NaN       NaN       NaN  NaN  NaN
B_N        NaN       NaN       NaN  NaN  NaN
B_P        NaN  0.106635       NaN  NaN  NaN
B_M+       NaN  0.105968       NaN  NaN  NaN
B_M-  0.099389  0.067714  0.099258  NaN  NaN


In [88]:
# Calculate monthly stock returns
r_monthly = r.resample('ME').apply(lambda x: np.nan if pd.isna(x).all() else (1 + x).prod() - 1)

In [89]:
# =============================================================================
#   3. Fama-MacBeth type predictive regression: CAPM
# =============================================================================

# Cross-sectional regression
lambda_0, lambda_CAPM, rsquared = [], [], []
models = []

for i in range(1, len(r_monthly)):
    temp_df = pd.DataFrame({'CAPM': beta_CAPM.iloc[i-1], 
                            'r': r_monthly.iloc[i]})
    temp_df = temp_df.dropna()
    x = sm.add_constant(temp_df[['CAPM']])
    y = temp_df['r']
    model = sm.OLS(y, x).fit()
    models.append(model)
    lambda_0.append(model.params['const'])
    lambda_CAPM.append(model.params['CAPM'])
    rsquared.append(model.rsquared)

# Results
lambdas = pd.DataFrame({'lambda_0': lambda_0, 'lambda_CAPM': lambda_CAPM})
mean_lambdas = lambdas.mean()

# Monthly risk premia estimate (%)
lambda_0_coef = mean_lambdas['lambda_0'] * 100
lambda_CAPM_coef = mean_lambdas['lambda_CAPM'] * 100

# Newwey-West t-statistic
nlags = round(0.75 * len(r_monthly)**(1/3))
nw_tstat0, p_values0 = newey_west_tstat(lambdas['lambda_0'], mean_lambdas['lambda_0'], nlags)
nw_tstat1, p_values1 = newey_west_tstat(lambdas['lambda_CAPM'], mean_lambdas['lambda_CAPM'], nlags)

# R-squared
rsquared_mean = np.mean(rsquared) * 100


# Print results
print(f'Constant: {lambda_0_coef:.2f} ({nw_tstat0[0]:.2f}) ({p_values0[0]:.2f})')
print(f'CAPM: {lambda_CAPM_coef:.2f} ({nw_tstat1[0]:.2f}) ({p_values1[0]:.2f})')
print('R^2:', rsquared_mean)

Constant: -1.66 (-1.03) (0.30)
CAPM: 0.60 (0.98) (0.33)
R^2: 2.7424254733190394


In [90]:
# =============================================================================
#   3. Fama-MacBeth type predictive regression: 4 Semibetas
# =============================================================================

# Cross-sectional regression
lambda_0, lambda_N, lambda_P, lambda_M_pos, lambda_M_neg, rsquared = [], [], [], [], [], []
for i in range(1, len(r_monthly)):
    temp_df = pd.DataFrame({'N': beta_N.iloc[i-1], 
                            'P': beta_P.iloc[i-1], 
                            'M_pos': beta_M_pos.iloc[i-1], 
                            'M_neg': beta_M_neg.iloc[i-1], 
                            'r': r_monthly.iloc[i]})
    temp_df = temp_df.dropna()
    x = sm.add_constant(temp_df[['N', 'P', 'M_pos', 'M_neg']])
    y = temp_df['r']
    model = sm.OLS(y, x).fit()
    lambda_0.append(model.params['const'])
    lambda_N.append(model.params['N'])
    lambda_P.append(model.params['P'])
    lambda_M_pos.append(model.params['M_pos'])
    lambda_M_neg.append(model.params['M_neg'])
    rsquared.append(model.rsquared)

# Results
lambdas = pd.DataFrame({'lambda_0': lambda_0, 'lambda_N': lambda_N, 'lambda_P': lambda_P, 'lambda_M_pos': lambda_M_pos, 'lambda_M_neg': lambda_M_neg})
mean_lambdas = lambdas.mean()

# Monthly risk premia estimate (%)
lambda_0_coef = mean_lambdas['lambda_0'] * 100
lambda_N_coef = mean_lambdas['lambda_N'] * 100
lambda_P_coef = mean_lambdas['lambda_P'] * 100
lambda_M_pos_coef = mean_lambdas['lambda_M_pos'] * 100
lambda_M_neg_coef = mean_lambdas['lambda_M_neg'] * 100

# Newwey-West t-statistic
nw_lag = round(0.75 * len(r_monthly)**(1/3))
nw_tstat0, p_values0 = newey_west_tstat(lambdas['lambda_0'], mean_lambdas['lambda_0'], nlags)
nw_tstat_N, p_values_N = newey_west_tstat(lambdas['lambda_N'], mean_lambdas['lambda_N'], nlags)
nw_tstat_P, p_values_P = newey_west_tstat(lambdas['lambda_P'], mean_lambdas['lambda_P'], nlags)
nw_tstat_M_pos, p_values_M_pos = newey_west_tstat(lambdas['lambda_M_pos'], mean_lambdas['lambda_M_pos'], nlags)
nw_tstat_M_neg, p_values_M_neg = newey_west_tstat(lambdas['lambda_M_neg'], mean_lambdas['lambda_M_neg'], nlags)

# R-squared
rsquared_mean = np.mean(rsquared) * 100

# Print results
print(f'Constant: {lambda_0_coef:.2f} ({nw_tstat0[0]:.2f}) ({p_values0[0]:.2f})')
print(f'N: {lambda_N_coef:.2f} ({nw_tstat_N[0]:.2f}) ({p_values_N[0]:.2f})')
print(f'P: {lambda_P_coef:.2f} ({nw_tstat_P[0]:.2f}) ({p_values_P[0]:.2f})')
print(f'M_pos: {lambda_M_pos_coef:.2f} ({nw_tstat_M_pos[0]:.2f}) ({p_values_M_pos[0]:.2f})')
print(f'M_neg: {lambda_M_neg_coef:.2f} ({nw_tstat_M_neg[0]:.2f}) ({p_values_M_neg[0]:.2f})')
print('R^2:', rsquared_mean)

Constant: -0.78 (-0.71) (0.48)
N: -0.61 (-0.45) (0.66)
P: -0.61 (-0.76) (0.44)
M_pos: -1.80 (-0.99) (0.32)
M_neg: -2.10 (-1.42) (0.16)
R^2: 5.922366547768163
